In [1]:
pip install tensorflow keras matplotlib scikit-learn opencv-python pandas

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

In [37]:
DATASET_DIR =r"C:\Users\acer\Downloads\age detection"  # Directory containing train, test folders, and age_detection.csv
TRAIN_DIR = os.path.join(DATASET_DIR, "train")
TEST_DIR = os.path.join(DATASET_DIR, "test")
IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 2

In [28]:
csv_path = os.path.join(DATASET_DIR, "age_detection.csv")
if os.path.exists(csv_path):
    metadata = pd.read_csv(csv_path)
    print(metadata.head())

                file    age  split
0  train/18-20/0.jpg  18-20  train
1  train/18-20/1.jpg  18-20  train
2  train/18-20/2.jpg  18-20  train
3  train/18-20/3.jpg  18-20  train
4  train/18-20/4.jpg  18-20  train


In [29]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    horizontal_flip=True,
    rotation_range=15,
    zoom_range=0.2
)

train_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_gen = datagen.flow_from_directory(
    DATASET_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 120 images belonging to 2 classes.
Found 30 images belonging to 2 classes.


In [32]:
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_gen = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

print("Data successfully loaded!")

Found 25 images belonging to 5 classes.
Data successfully loaded!


In [38]:
def build_baseline_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    model = Sequential([
        base_model,
        Flatten(),
        Dense(256, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])
    return model

baseline_model = build_baseline_model((IMG_SIZE, IMG_SIZE, 3), NUM_CLASSES)
baseline_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [39]:
baseline_history = baseline_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    verbose=1
)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 123s 18s/step - accuracy: 0.7114 - loss: 36.3316 - val_accuracy: 0.8333 - val_loss: 53.0422
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 54s 13s/step - accuracy: 0.8542 - loss: 0.4886 - val_accuracy: 0.8333 - val_loss: 1.2317
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 56s 15s/step - accuracy: 0.8066 - loss: 0.6594 - val_accuracy: 0.8333 - val_loss: 0.6917
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 56s 15s/step - accuracy: 0.8551 - loss: 0.6698 - val_accuracy: 0.8333 - val_loss: 0.6896
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 63s 16s/step - accuracy: 0.8289 - loss: 0.6358 - val_accuracy: 0.8333 - val_loss: 0.6872
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 51s 12s/step - accuracy: 0.8427 - loss: 0.6149 - val_accuracy: 0.8333 - val_loss: 0.6849
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 50s 13s/step - accuracy: 0.8409 - loss: 0.5993 - val_accuracy: 0.8333 - val_loss: 76.7543
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 51s 13s/step - accuracy: 0.7958 - loss: 0.7822 - val_accuracy: 0.8333 - val_loss: 66.

In [40]:
baseline_eval = baseline_model.evaluate(val_gen)
print("Baseline Model Evaluation:", baseline_eval)

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.8333 - loss: 33.2377
Baseline Model Evaluation: [33.237667083740234, 0.8333333134651184]


In [41]:
def build_enhanced_model(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze base model layers
    model = Sequential([
        base_model,
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')  # Match number of classes
    ])
    return model

enhanced_model = build_enhanced_model((IMG_SIZE, IMG_SIZE, 3), NUM_CLASSES)
enhanced_model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])


In [42]:
enhanced_history = enhanced_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    verbose=1
)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 41s 8s/step - accuracy: 0.4843 - loss: 1.7514 - val_accuracy: 0.1667 - val_loss: 8.1511
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 23s 6s/step - accuracy: 0.4975 - loss: 1.1873 - val_accuracy: 0.1667 - val_loss: 7.4628
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 6s/step - accuracy: 0.5070 - loss: 1.3983 - val_accuracy: 0.1667 - val_loss: 6.3001
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.6169 - loss: 0.9299 - val_accuracy: 0.1667 - val_loss: 4.3180
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 22s 5s/step - accuracy: 0.5440 - loss: 1.0255 - val_accuracy: 0.1667 - val_loss: 4.1028
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 23s 6s/step - accuracy: 0.5863 - loss: 1.0959 - val_accuracy: 0.1667 - val_loss: 3.0283
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 23s 6s/step - accuracy: 0.6022 - loss: 0.9634 - val_accuracy: 0.1667 - val_loss: 2.3827
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 23s 6s/step - accuracy: 0.6515 - loss: 0.7487 - val_accuracy: 0.2000 - val_loss: 1.3691
Epoch 9/

In [43]:
enhanced_eval = enhanced_model.evaluate(val_gen)
print("Enhanced Model Evaluation:", enhanced_eval)

1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.8333 - loss: 1.0531
Enhanced Model Evaluation: [1.053065538406372, 0.8333333134651184]


In [44]:
def evaluate_model(model, data_gen):
    predictions = model.predict(data_gen)
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = data_gen.classes
    class_labels = list(data_gen.class_indices.keys())

    print(classification_report(true_classes, predicted_classes, target_names=class_labels))
    print("Confusion Matrix:\n", confusion_matrix(true_classes, predicted_classes))

print("\nBaseline Model Metrics:")
evaluate_model(baseline_model, val_gen)

print("\nEnhanced Model Metrics:")
evaluate_model(enhanced_model, val_gen)


Baseline Model Metrics:
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
              precision    recall  f1-score   support

        test       0.00      0.00      0.00         5
       train       0.83      1.00      0.91        25

    accuracy                           0.83        30
   macro avg       0.42      0.50      0.45        30
weighted avg       0.69      0.83      0.76        30


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



Confusion Matrix:
 [[ 0  5]
 [ 0 25]]

Enhanced Model Metrics:
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
              precision    recall  f1-score   support

        test       0.00      0.00      0.00         5
       train       0.83      1.00      0.91        25

    accuracy                           0.83        30
   macro avg       0.42      0.50      0.45        30
weighted avg       0.69      0.83      0.76        30

Confusion Matrix:
 [[ 0  5]
 [ 0 25]]


c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [45]:
baseline_model.save("baseline_model.h5")
enhanced_model.save("enhanced_model.h5")

In [46]:
for x_batch, y_batch in train_gen:
    print(f"Input batch shape: {x_batch.shape}, Target batch shape: {y_batch.shape}")
    break


Input batch shape: (32, 224, 224, 3), Target batch shape: (32, 2)
